Background Correction demonstration.
After some trial and error, different methods for background correction were researched and developed. They are demonstrated in this notebook. For further explanation, see the report 'Supporting information to the background correction code'.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
from matplotlib.animation import FuncAnimation
import time
import pandas as pd

from Particledrop import Landing_Flashes

import trackpy

Call this function to execute an animation. Change the interval value to change the speed

In [ ]:
def execute_amovie(moviedata):
    plt.rcParams["animation.html"] = "jshtml"
    plt.rcParams['figure.dpi'] = 100  
    plt.ioff()
    
    fig = plt.figure()
    axis = plt.axes()
    
    film = plt.imshow(moviedata[0])
    
    def init(): 
        film.set_data(moviedata[0])
        return [film]
    
    def animate(i):
        film.set_array(moviedata[i])
        return [film]
    
    anim = FuncAnimation(fig, animate, frames = moviedata.shape[0], init_func = init, interval = 200, blit = True)

    return anim

First, define all the operating functions on the dataset:

In [ ]:
#Do simple backgroundcorrection
#Simple mode: Take off the reference (= median), divide by reference - dark
#Dark value is now set to 5 in the simulation, could be a variable.
#If we put it to 5 here as well, you can get a divide by 0 if there's no noise
def Simple_backgroundcor(mov):
    bg = np.median(mov,axis=0)

    backgroundcor = np.zeros((nf,mov[0].shape[0],mov[0].shape[1]))

    dark = np.ones((fov))*5
    for i in range(nf):
        backgroundcor[i] = (mov[i] - bg)/(bg-dark)

    #Set negative values to 0 
    backgroundcor[backgroundcor<0] = 0  
    
    return backgroundcor
#The moving average takes off the mean of the past few frames.
#Very usefull in combination with trackpy batch and linking

def MovingAverage(movie):
    start = 6
    end = nf

    BGCma = movie[start:end,:,:] - np.mean([movie[start-5:end-5,:,:],
                                          movie[start-4:end-4,:,:],
                                          movie[start-3:end-3,:,:],
                                          movie[start-2:end-2,:,:],
                                          movie[start-1:end-1,:,:]],
                                          axis = 0)
    #Normalization could be important
    BGCma2 = BGCma/movie[0]
    return BGCma2

#Or, if you just want to subtract the last frame:

def MovingAverageOneFrame(movie):
    startframe = 0
    endframe = nf 
    startframe2 = 1 # +1 for accounting for the moving average
    endframe2 = endframe-startframe

    BGCma = movie[startframe2:endframe2,:,:] - np.mean([movie[startframe2-1:endframe2-1,:,:]], axis = 0)
    
    BGCma2 = BGCma/movie[0]
    return BGCma2

#This function makes a Histogram of all intensities in a movie. Use this 
def Histogram_intensities(movie):
    movielist = movie.flatten()

    fig, ax = plt.subplots()
    plt.title('Intensities in movie:')
    ax.hist(movielist, bins = 30)
    
def Cutoff_movie(movie, cutoff):
    movie_cutoff = np.where(movie < cutoff, 0, movie)
    return movie_cutoff

#For the Fourier transform, it's good to start with images before doing the entire movie
#These are functions for Fourier transforming and plotting an image
from scipy import fftpack
def fft_img(img):
    im_fft = fftpack.fft2(img)
    return im_fft

def plot_spectrum(im_fft):
    from matplotlib.colors import LogNorm
    # A logarithmic colormap
    plt.figure()
    plt.title('Fourier transform')
    plt.imshow(np.abs(im_fft), norm=LogNorm(vmin=5))
    plt.colorbar()

#This function cuts off a lot of the fft, it keeps a given fraction in the corners (stolen from internet)
#and returns the reconstructed image from that cutoff

# Define the fraction of coefficients (in each direction) we keep: keep_fraction

def fft_cutoff_plots(img, keep_fraction):
    plt.figure()
    plt.imshow(img)
    plt.title('Normal image')
    
    im_fft = fftpack.fft2(img)
    
    plt.figure()
    plot_spectrum(im_fft)
    plt.title('Fourier Transformed Spectrum')
    
    # In the lines following, we'll make a copy of the original spectrum and
    # truncate coefficients.
    
    # Call ff a copy of the original transform. Numpy arrays have a copy
    # method for this purpose.
    im_fft2 = im_fft.copy()

    # Set r and c to be the number of rows and columns of the array.
    r, c = im_fft2.shape

    # Set to zero all rows with indices between r*keep_fraction and
    # r*(1-keep_fraction):
    im_fft2[int(r*keep_fraction) : int(r*(1-keep_fraction))] = 0

    # Similarly with the columns:
    im_fft2[:, int(c*keep_fraction) : int(c*(1-keep_fraction))] = 0

    plt.figure()
    plot_spectrum(im_fft2)
    plt.title('Filtered Spectrum')
    
    # Reconstruct the denoised image from the filtered spectrum, keep only the
    # real part for display.
    im_new = fftpack.ifft2(im_fft2).real

    plt.figure()
    plt.imshow(im_new)
    plt.title('Reconstructed Image')
    return im_new

def fft_cutoff_noplot(img, keep_fraction):
    im_fft = fftpack.fft2(img)
    im_fft2 = im_fft.copy()
    r, c = im_fft2.shape
    im_fft2[int(r*keep_fraction):int(r*(1-keep_fraction))] = 0
    im_fft2[:, int(c*keep_fraction):int(c*(1-keep_fraction))] = 0
    im_new = fftpack.ifft2(im_fft2).real
    return im_new
    
def fft_cutoff_movie(movie, keep_fraction):
    keepfraction = keep_fraction
    movie_new = []
    for i in range(movie.shape[0]):
        frame = movie[i,:,:]
        frame_new = fft_cutoff_noplot(frame, keepfraction)
        movie_new.append(frame_new)
    movie_new = np.asarray(movie_new)
    return movie_new

#And the trackpy function which uses batch and link:
#Trackpy batch analyses all frames, link looks for particles over multiple frames, 
#stubs filters for particles apparant for a certain number of frames


def trackpy_movie(movie, size, mmass):
    nf = movie.shape[0]
    f1 = trackpy.batch(movie[:,:,:], size, minmass=mmass, invert=False, processes='auto')
    t = trackpy.link(f1, 5, memory=0)
    
    t1 = trackpy.filter_stubs(t, 3)

    t2 = t1.rename(columns={'frame':'Frame'})
    particles = t2['particle'].nunique()
    print('Unique particles found:', + particles)
    
    # t2.groupby('Frame')['particle'].nunique().plot(kind='line')
    # plt.yticks(np.arange(0, particles+1, 5.0))
    # plt.xticks(np.arange(0, nf+1, 10.0))
    # plt.title('Number of landed particles over the series of frames')
    # plt.xlabel('Frame number')
    # plt.ylabel('Number of particles')
    # plt.grid()
    # plt.show()
    
    nparticles = []
    frames = np.arange(0,nf,1)

    for i in range(nf):
        temp = t2.loc[t2['Frame'] == i]
        nparticles.append(len(temp))

    plt.figure()
    plt.step(frames,nparticles)
    plt.yticks(np.arange(0, particles+1, 5.0))
    plt.xticks(np.arange(0, nf+1, 10.0))
    plt.title('Stepfunction number of landed particles over the series of frames')
    plt.xlabel('Frame number')
    plt.ylabel('Number of particles')
    plt.grid()
    plt.show()
    
    t_hist = t2.drop_duplicates(subset=['particle'])
    plt.title('Found Mass Histogram')
    plt.hist(t_hist['mass'])
    
    plt.figure()
    trackpy.annotate(t2[t2['Frame'] == nf-1], movie[nf-1]);
    
#     return t2, t_hist
    return

#For a moving average, the filter should be lower
#And the cumulative graph should be made differently
def trackpy_movie_movav(movie, size, mmass):
    nf = movie.shape[0]
    f1 = trackpy.batch(movie[:,:,:], size, minmass=mmass, invert=False, processes='auto')
    t = trackpy.link(f1, 5, memory=0)
    
    t1 = trackpy.filter_stubs(t, 1)

    t2 = t1.rename(columns={'frame':'Frame'})
    particles = t2['particle'].nunique()
    print('Unique particles found:', + particles)
    
    # t2.groupby('Frame')['particle'].nunique().plot(kind='line')
    # plt.yticks(np.arange(0, particles+1, 5.0))
    # plt.xticks(np.arange(0, nf+1, 10.0))
    # plt.title('Number of landed particles over the series of frames')
    # plt.xlabel('Frame number')
    # plt.ylabel('Number of particles')
    # plt.grid()
    # plt.show()
    
    nparticles = []
    frames = np.arange(0,nf,1)

    for i in range(nf):
        temp = t2.loc[t2['Frame'] == i]
        nparticles.append(len(temp))

    plt.figure()
    plt.step(frames,nparticles)
    plt.yticks(np.arange(0, particles+1, 5.0))
    plt.xticks(np.arange(0, nf+1, 10.0))
    plt.title('Stepfunction number of landed particles over the series of frames')
    plt.xlabel('Frame number')
    plt.ylabel('Number of particles')
    plt.grid()
    plt.show()
    
    t_hist = t2.drop_duplicates(subset=['particle'])
    plt.title('Found Mass Histogram')
    plt.hist(t_hist['mass'])
    
    plt.figure()
    trackpy.annotate(t2, movie[nf-1]);
    
#     return t2, t_hist
    return

Now that's done, let's make a dataset which we'll test the different functions on. Note that the noise is changed in meas.noise=20. This is a property of the fact that the meas object is a class, and different parameters can be changed afterwards like that. If you want to change the noise after this, be sure to also make a new plis and movie dataset. In this notebook, the noise is kept at level 20, so all the methods track the particles quite well. The limits of the methods are mentioned with each demonstrated method. 

In [ ]:
nf = 80
fov=[200,300]
meas = Landing_Flashes(fov=fov, seed=5045, numpar = 50, noise = 0, nframes = nf, signal = 20, sizevar=0.2, dark = 5, psize = 4, unevenIllumination = False)
testframe = nf-1
meas.noise = 20
plis = meas.parlist
movie = meas.genStack()
print('Number of particles simulated:', np.shape(plis)[0])

plt.figure()
plt.title('Simulated Size histogram')
plt.hist(plis[:,2])

In [ ]:
execute_amovie(movie)

The written trackpy_movie fucntion tries to find all particles. The inputs are size and minmass, two parameters of the trackpy function. Size is the estimation of how big the particles are, minmass the minimal intensity they need to have for trackpy to track them. Note that all particles are found, except ones that land over each other. 


In [ ]:
trackpy_movie(movie, 7, 40)

When a moving average is taken, the particles become flashes.

In [ ]:
MovAv = MovingAverage(movie)
execute_amovie(MovAv)

The trackpy plot does not yet work as planned for this moving average, but it has found the double particles!

In [ ]:
trackpy_movie_movav(MovAv, 9, 7)

Or just take 1 frame off: Then the plots can be fixed! (Ingmar, can you look at that? It was supposed to be in I_Overview right? The cumulative plot?)

In [ ]:
MovAv1 = MovingAverageOneFrame(movie)

In [ ]:
print(MovAv1.shape)

In [ ]:
particlelist = []
array = []
for i in np.arange(0, MovAv1.shape[0]):
    plt.title(f'Trackpy locate on frame no.{i}')
    f = trackpy.locate(MovAv1[i, :, :], 9, minmass=7, invert=False) # 9 = the est. pixel size, minmass = intensity 
    trackpy.annotate(f, MovAv1[i, :, :])
    particleframe = len(f['mass'])
    if len(f['mass']) != 0:
        print('A non-zero value!, namely '+str(particleframe))
        particlelist.append(len(f['mass']))
        array.append(i)
    else:
        print('No particles located in this frame!')

allparticles = sum(particlelist)
print('The total number of landed particles = '+str(allparticles))
print(particlelist)
print(array)

In [ ]:
# print(array)  # on which frame are particles found
print(sum(particlelist))  # number of counts
cdf = np.cumsum(particlelist)
# plt.figure(figsize=(20,20))  # use larger figsize if the number of frames is high
plt.step(array, cdf)
plt.title('Cumulative number of landed particles over the series of frames')
plt.xlabel('Frame number')
plt.ylabel('Number of particles')
plt.xticks(np.arange(0, max(array)+1, 5.0))
plt.yticks(np.arange(0, max(cdf)+1, 1.0))
plt.grid()
plt.show()

Now for the Cutoff. A histogram of all intensities is made, and the lower intensities are cut off as a backgroundcorrection. The max noise for which this works seems to be around 20, which is less than just using trackpy, but with higher noises finding the cutoff doesn't work and you delete both signal and noise. At noise level 20, this is already iffy, trackpy does not find all particles.
Here you see that the cutoff should be around 30. The Cutoff_movie function does this. 

In [ ]:
Histogram_intensities(movie)

In [ ]:
movie_cutoff = Cutoff_movie(movie, 30)
trackpy_movie(movie_cutoff,7,80)

The FFT method. First, try out the parameters on a single testframe. The function fft_cutoff_plots gives the fourier transformed spectrum, cut off spectrum and reconstructed image from the cut off spectrum.

In [ ]:
img = movie[testframe,:,:]

#Remember, the number you put into this function is the fraction to keep
fft_cutoff_plots(img,0.11)

Now, put in these parameters to transform the whole movie. This works very well. Combine this with a working moving average to find all particles.

In [ ]:
moviefft = fft_cutoff_movie(movie, 0.11)
execute_amovie(moviefft)

In [ ]:
trackpy_movie(moviefft, 7, 60)